In [ ]:
import h5py
from os.path import join
from mssa_learning.preprocessing import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import progressbar
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from mssa_learning.models.dnn_classifier import DNNClassifier
from model_generator.posture_classification.rnn_classifier import RNNClassifier
import math

from sklearn.svm import LinearSVC
import csv

In [ ]:
def create_training_base(features, targets, nb_targets=None):
    if nb_targets is None:
        nb_targets = len(targets)

    idx = np.arange(len(features))
    np.random.shuffle(idx)
    nb_samples = int(9*len(idx)/(10*nb_targets))
    samples_per_class = np.zeros(nb_targets)
    max_samples = np.ones(nb_targets) * nb_samples

    i = 0
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    while not np.equal(samples_per_class, max_samples).all() and i<len(targets):
        if samples_per_class[targets[i]] < nb_samples:
            train_x.append(features[i])
            train_y.append(targets[i])
            samples_per_class[targets[i]] += 1
        else:
            test_x.append(features[i])
            test_y.append(targets[i])
        i += 1

    for j in range(i, len(targets)):
        test_x.append(features[j])
        test_y.append(targets[j])
    return np.array(train_x), np.array(train_y), np.array(test_x), np.array(test_y)

In [ ]:
init_notebook_mode(connected=True)

In [ ]:
h5_pc = h5py.File(join("..", "data", "dataset_converted" , "h5", "pc.h5"))

In [ ]:
pcs = []
labels = []
nb_components = 10
nb_labels = 5

for key in h5_pc:
    for rec in h5_pc[key]:
        if not math.isnan(h5_pc[key][rec][0][0]):
            label = int(key[-3:]) - 1
            if label < nb_labels:
                labels.append(label)
                pcs.append(h5_pc[key][rec][:, :nb_components])

In [ ]:
pcs, size = dynamic_time_warping(pcs, principal_components_distance)

In [ ]:
flat_pcs = []
flat_labels = []
for i, pc in enumerate(pcs):
    if len(pc) == size: 
        flat_pcs.append([x for row in pc for x in row])
        flat_labels.append(labels[i])

In [ ]:
train_x, train_y, test_x, test_y = create_training_base(flat_pcs, flat_labels, nb_labels)

#### SVM Classification

In [ ]:
clf = LinearSVC(random_state=0, dual=False)

In [ ]:
clf.fit(train_x, train_y)

In [ ]:
clf.score(test_x, test_y)

#### DNN Classification

In [ ]:
D_in = len(flat_pcs[0])
H = 64
D_out = nb_labels  # number of classes to learn

In [ ]:
batch = 100
steps = 12000
epochs = 2

In [ ]:
model = DNNClassifier(D_in, D_out, H, batch=batch, steps=steps, epochs=epochs, scale=False)

In [ ]:
model.fit(train_x, train_y)

In [ ]:
nb_errors = 0
for i, x in enumerate(test_x):
    y = torch.from_numpy(x).float()
    outputs = model.forward(Variable(y))
    # value, index = torch.max(outputs.data, 0, keepdim=True)
    y_pred = int(outputs.data.max(0)[1])
    if y_pred != test_y[i]:
        nb_errors += 1
success_rate = (len(test_x) - nb_errors) / float(len(test_x))
print success_rate

#### RNN Classification

In [ ]:
pcs = []
labels = []
nb_components = 10
nb_labels = 5

for key in h5_pc:
    for rec in h5_pc[key]:
        if not math.isnan(h5_pc[key][rec][0][0]):
            label = int(key[-3:]) - 1
            if label < nb_labels:
                labels.append(label)
                pcs.append(h5_pc[key][rec][:, :nb_components])

In [ ]:
train_x, train_y, test_x, test_y = create_training_base(pcs, labels, nb_labels)

In [ ]:
D_in = 10
H = 16
D_out = max(labels)  # number of classes to learn

In [ ]:
steps = 50000
epochs = 1

In [ ]:
model = RNNClassifier(D_in, D_out, H, steps=steps, epochs=epochs, scale=True)

In [ ]:
model.fit(train_x, train_y)

In [ ]:
nb_errors = 0
iter_example = 1000
for i in range(iter_example):
    x, y = model.random_example(test_x, test_y)
    hidden = model.initHidden()
    for i in range(x.size()[0]):
        output, hidden = model.forward(x[i], hidden)
    # value, index = torch.max(outputs.data, 0, keepdim=True)
    top_n, top_i = output.data.topk(1)
    y_pred = int(top_i[0][0])
    if y_pred != int(y.data[0]):
        nb_errors += 1
success_rate = (iter_example - nb_errors) / float(iter_example)
print success_rate

#### CSV export

In [ ]:
def data_to_rows(features, labels):
    rows = np.hstack((features, labels))
    return rows

In [ ]:
with open('/tmp/train.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerows(rows)